In [1]:
pip install pandas sqlalchemy psycopg2


In [2]:
!pip install pandas sqlalchemy psycopg2-binary


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 43.4 MB/s eta 0:00:00


In [5]:
from google.colab import files
uploaded = files.upload()   # choose api_data.csv and flatfile_data.csv


Saving billing_data.csv to billing_data.csv
Saving patients_data.csv to patients_data.csv
Saving treatments_data.csv to treatments_data.csv
Saving insurance_clean.csv to insurance_clean (1).csv
Saving api_health_data_clean.csv to api_health_data_clean (1).csv


**EXTRACT**


In [6]:
import pandas as pd
from sqlalchemy import create_engine

# OLTP DATABASES (PostgreSQL)
def extract_patient_data():
    return pd.read_csv("/content/patients_data.csv")

def extract_billing_data():
    return pd.read_csv("/content/billing_data.csv")

def extract_treatments_data():
    return pd.read_csv("/content/treatments_data.csv")


#  API (CSV already fetched)
def extract_api_data():
    # adjust path if files are uploaded or in Drive
    return pd.read_csv("/content/api_health_data_clean (1).csv")

#  FLAT FILE (CSV / Parquet)
def extract_flatfile_data():
    return pd.read_csv("/content/insurance_clean (1).csv")

#   TESTING EXTRACTION — sample outputs


try:
    patients = extract_patient_data()
    print("✅ Patient data extracted:", patients.shape)
except Exception as e:
    print("⚠️ Skipping DB connection (set up later) —", e)
    patients = pd.DataFrame()

try:
    billing = extract_billing_data()
    print("✅ Billing data extracted:", billing.shape)
except Exception as e:
    print("⚠️ Skipping DB connection (set up later) —", e)
    billing = pd.DataFrame()

try:
    treatments = extract_treatments_data()
    print("✅ Treatments data extracted:", treatments.shape)
except Exception as e:
    print("⚠️ Skipping DB connection (set up later) —", e)
    treatments = pd.DataFrame()

# CSV sources
api = extract_api_data()
flatfile = extract_flatfile_data()

print("\nAPI Data preview:\n", api.head())
print("\nFlat File preview:\n", flatfile.head())


✅ Patient data extracted: (55500, 16)
✅ Billing data extracted: (163065, 13)
✅ Treatments data extracted: (200, 6)

API Data preview:
   CountryCode  Year                    Indicator_Name  ValueNumeric
0         BHR  2011  Life expectancy at birth (years)     75.205359
1         DNK  2014  Life expectancy at birth (years)     78.616875
2         MAR  2020  Life expectancy at birth (years)     73.869852
3         JOR  2014  Life expectancy at birth (years)     78.390969
4         URY  2015  Life expectancy at birth (years)     77.099252

Flat File preview:
    age     sex     bmi  children smoker     region      charges age_group  \
0   19  female  27.900         0    yes  southwest  16884.92400     19-30   
1   18    male  33.770         1     no  southeast   1725.55230     19-30   
2   28    male  33.000         3     no  southeast   4449.46200     19-30   
3   33    male  22.705         0     no  northwest  21984.47061     31-45   
4   32    male  28.880         0     no  northwest 

In [51]:
# Check if the file loaded properly
print("Treatments shape:", treatments.shape)
print(treatments.head())

# If empty, reload explicitly
if treatments.empty:
    treatments = pd.read_csv("/content/treatments_data.csv")  # adjust path if needed
    print("Reloaded treatments file:", treatments.shape)
    print(treatments.head())


Treatments shape: (0, 9)
Empty DataFrame
Columns: [treatment_id, appointment_id, treatment_type, description, cost, treatment_date, treatment_year, treatment_month, patient_id]
Index: []
Reloaded treatments file: (200, 6)
  treatment_id appointment_id treatment_type         description     cost  \
0         T001           A001   Chemotherapy     Basic screening  3941.97   
1         T002           A002            MRI   Advanced protocol  4158.44   
2         T003           A003            MRI  Standard procedure  3731.55   
3         T004           A004            MRI     Basic screening  4799.86   
4         T005           A005            ECG  Standard procedure   582.05   

  treatment_date  
0     2023-08-09  
1     2023-06-09  
2     2023-06-28  
3     2023-09-01  
4     2023-07-06  


**TRANSFORM**

In [52]:
import pandas as pd
import numpy as np

# =====================================================
# ✅ 1️⃣ NORMALIZE COLUMN NAMES
# =====================================================
for df in (patients, billing, treatments, api, flatfile):
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# =====================================================
# 🟩 DIM_PATIENT
# =====================================================
dim_patient = patients[[
    "patient_id", "name", "age", "gender", "blood_type", "medical_condition",
    "insurance_provider", "hospital", "doctor", "billing_amount"
]].copy()

# Merge lifestyle/charges info from flatfile if available
flat_cols = ["age", "sex", "bmi", "smoker", "region", "charges"]
flat_enrich = flatfile[[c for c in flat_cols if c in flatfile.columns]].copy()
if 'sex' in flat_enrich.columns:
    flat_enrich.rename(columns={'sex': 'gender'}, inplace=True)

available_cols = [c for c in ["age", "gender", "bmi", "smoker", "region", "charges"] if c in flat_enrich.columns]
dim_patient = dim_patient.merge(flat_enrich[available_cols], on="age", how="left")

dim_patient["patient_key"] = ["P" + str(i+1).zfill(4) for i in range(len(dim_patient))]

final_patient_cols = ["patient_key", "patient_id", "name", "age", "blood_type",
                      "medical_condition", "insurance_provider", "region", "smoker", "bmi", "charges"]
if 'gender' in dim_patient.columns:
    final_patient_cols.insert(4, 'gender')

dim_patient = dim_patient[final_patient_cols]

# =====================================================
# 🟨 DIM_DOCTOR
# =====================================================
dim_doctor = patients[["doctor", "hospital", "room_number", "admission_type"]].drop_duplicates().copy()
dim_doctor["doctor_key"] = ["D" + str(i+1).zfill(4) for i in range(len(dim_doctor))]
dim_doctor.rename(columns={"doctor": "doctor_name"}, inplace=True)
dim_doctor = dim_doctor[["doctor_key", "doctor_name", "hospital", "room_number", "admission_type"]]

# =====================================================
# 🟧 DIM_HOSPITAL
# =====================================================
dim_hospital = billing[[
    "provider_street_address", "provider_city", "provider_state",
    "provider_zip_code", "hospital_referral_region_description"
]].drop_duplicates().copy()
dim_hospital["hospital_key"] = ["H" + str(i+1).zfill(4) for i in range(len(dim_hospital))]
dim_hospital = dim_hospital[[
    "hospital_key", "provider_street_address", "provider_city",
    "provider_state", "provider_zip_code", "hospital_referral_region_description"
]]

# =====================================================
# 🟪 DIM_TREATMENT
# =====================================================
dim_treatment = treatments.copy()
dim_treatment["treatment_key"] = ["T" + str(i+1).zfill(4) for i in range(len(dim_treatment))]

# Parse treatment_date
dim_treatment["treatment_date"] = pd.to_datetime(dim_treatment["treatment_date"], errors="coerce")
dim_treatment["treatment_year"] = dim_treatment["treatment_date"].dt.year
dim_treatment["treatment_month"] = dim_treatment["treatment_date"].dt.month

dim_treatment = dim_treatment[[
    "treatment_key", "treatment_id", "treatment_type", "description", "cost", "treatment_date"
]]

# =====================================================
# 🟥 DIM_DATE
# =====================================================
date_cols = []
for col in ["date_of_admission", "discharge_date", "treatment_date"]:
    if col in patients.columns:
        date_cols.append(patients[col])
    if col in treatments.columns:
        date_cols.append(treatments[col])

if date_cols:
    all_dates = pd.concat(date_cols)
    dim_date = pd.DataFrame({"date": pd.to_datetime(all_dates.dropna().unique())})
    dim_date["day"] = dim_date["date"].dt.day
    dim_date["month"] = dim_date["date"].dt.month
    dim_date["quarter"] = dim_date["date"].dt.quarter
    dim_date["year"] = dim_date["date"].dt.year
    dim_date["date_key"] = ["DT" + str(i+1).zfill(4) for i in range(len(dim_date))]
else:
    dim_date = pd.DataFrame(columns=["date_key", "date", "day", "month", "quarter", "year"])

# =====================================================
# 🟫 DIM_HEALTH_INDICATOR (SAFE VERSION)
# =====================================================
api_cols = list(api.columns)
col_country = next((c for c in api_cols if "country" in c), None)
col_year = next((c for c in api_cols if "year" in c), None)
col_indicator = next((c for c in api_cols if "indicator" in c or "name" in c or "measure" in c), None)
col_value = next((c for c in api_cols if "value" in c or "numeric" in c or "val" in c), None)

if col_country and col_year and col_value:
    use_cols = [col_country, col_year]
    if col_indicator:
        use_cols.append(col_indicator)
    use_cols.append(col_value)

    dim_health_indicator = api[use_cols].copy()
    rename_map = {col_country: "country_code", col_year: "year", col_value: "value_numeric"}
    if col_indicator:
        rename_map[col_indicator] = "indicator_name"
    else:
        dim_health_indicator["indicator_name"] = pd.NA

    dim_health_indicator.rename(columns=rename_map, inplace=True)
    for c in ["country_code", "year", "indicator_name", "value_numeric"]:
        if c not in dim_health_indicator.columns:
            dim_health_indicator[c] = pd.NA

    dim_health_indicator.insert(0, "indicator_key", ["I" + str(i+1).zfill(4) for i in range(len(dim_health_indicator))])
    dim_health_indicator = dim_health_indicator[["indicator_key", "country_code", "year", "indicator_name", "value_numeric"]]
    print("✅ DimHealthIndicator created:", dim_health_indicator.shape)
else:
    print("⚠️ Skipping DimHealthIndicator (missing columns).")
    dim_health_indicator = pd.DataFrame(columns=["indicator_key", "country_code", "year", "indicator_name", "value_numeric"])

# =====================================================
# 🟦 FACT_PATIENT_TREATMENT
# =====================================================
if len(dim_treatment) == 0:
    print("⚠️ No treatment data found, skipping fact table generation.")
    fact_patient_treatment = pd.DataFrame()
else:
    np.random.seed(42)
    sampled_patients = dim_patient.sample(len(dim_treatment), replace=True).reset_index(drop=True)
    sampled_doctors = dim_doctor.sample(len(dim_treatment), replace=True).reset_index(drop=True)
    sampled_hospitals = dim_hospital.sample(len(dim_treatment), replace=True).reset_index(drop=True)

    fact_patient_treatment = pd.concat([
        dim_treatment.reset_index(drop=True),
        sampled_patients[["patient_key", "charges"]],
        sampled_doctors[["doctor_key"]],
        sampled_hospitals[["hospital_key"]]
    ], axis=1)

    fact_patient_treatment["fact_id"] = ["F" + str(i+1).zfill(4) for i in range(len(fact_patient_treatment))]
    fact_patient_treatment["billing_amount"] = fact_patient_treatment["charges"] * 0.8
    fact_patient_treatment["length_of_stay"] = np.random.randint(1, 10, size=len(fact_patient_treatment))
    fact_patient_treatment["total_discharges"] = np.random.randint(10, 100, size=len(fact_patient_treatment))
    fact_patient_treatment["average_covered_charges"] = np.random.uniform(1000, 5000, len(fact_patient_treatment))
    fact_patient_treatment["average_total_payments"] = np.random.uniform(500, 4000, len(fact_patient_treatment))
    fact_patient_treatment["average_medicare_payments"] = np.random.uniform(300, 2000, len(fact_patient_treatment))

    fact_patient_treatment = fact_patient_treatment[[
        "fact_id", "treatment_key", "treatment_id", "treatment_type", "description",
        "cost", "treatment_date", "patient_key", "doctor_key", "hospital_key",
        "billing_amount", "length_of_stay", "total_discharges",
        "average_covered_charges", "average_total_payments", "average_medicare_payments"
    ]]

print("\n✅ TRANSFORMATION COMPLETE!")
print("Shapes:")
print("DimPatient:", dim_patient.shape)
print("DimDoctor:", dim_doctor.shape)
print("DimHospital:", dim_hospital.shape)
print("DimTreatment:", dim_treatment.shape)
print("DimDate:", dim_date.shape)
print("DimHealthIndicator:", dim_health_indicator.shape)
print("FactPatientTreatment:", fact_patient_treatment.shape)


✅ DimHealthIndicator created: (7056, 5)

✅ TRANSFORMATION COMPLETE!
Shapes:
DimPatient: (1103987, 12)
DimDoctor: (50000, 5)
DimHospital: (3337, 6)
DimTreatment: (200, 6)
DimDate: (2015, 6)
DimHealthIndicator: (7056, 5)
FactPatientTreatment: (200, 16)


**LOAD**

In [54]:
output_path = "/content/clean_csvs/"  # change if needed
import os
os.makedirs(output_path, exist_ok=True)

dim_patient.to_csv(output_path + "dim_patient.csv", index=False)
dim_doctor.to_csv(output_path + "dim_doctor.csv", index=False)
dim_hospital.to_csv(output_path + "dim_hospital.csv", index=False)
dim_treatment.to_csv(output_path + "dim_treatment.csv", index=False)
dim_date.to_csv(output_path + "dim_date.csv", index=False)
dim_health_indicator.to_csv(output_path + "dim_health_indicator.csv", index=False)
fact_patient_treatment.to_csv(output_path + "fact_patient_treatment.csv", index=False)

# =====================================================
# 🧾 SUMMARY
# =====================================================
print("\n✅ TRANSFORMATION COMPLETE — SHAPES:")
print("DimPatient:", dim_patient.shape)
print("DimDoctor:", dim_doctor.shape)
print("DimHospital:", dim_hospital.shape)
print("DimTreatment:", dim_treatment.shape)
print("DimDate:", dim_date.shape)
print("DimHealthIndicator:", dim_health_indicator.shape)
print("FactPatientTreatment:", fact_patient_treatment.shape)

print("\n📊 Fact Table Preview:")
display(fact_patient_treatment.head())


✅ TRANSFORMATION COMPLETE — SHAPES:
DimPatient: (1103987, 12)
DimDoctor: (50000, 5)
DimHospital: (3337, 6)
DimTreatment: (200, 6)
DimDate: (2015, 6)
DimHealthIndicator: (7056, 5)
FactPatientTreatment: (200, 16)

📊 Fact Table Preview:


,fact_id,treatment_key,treatment_id,treatment_type,description,cost,treatment_date,patient_key,doctor_key,hospital_key,billing_amount,length_of_stay,total_discharges,average_covered_charges,average_total_payments,average_medicare_payments
0,F0001,T0001,T001,Chemotherapy,Basic screening,3941.97,2023-08-09,P121959,D46844,H1013,42072.663512,5,10,4154.388845,3083.117657,349.720381
1,F0002,T0002,T002,MRI,Advanced protocol,4158.44,2023-06-09,P671156,D31066,H1665,2830.162400,6,17,1606.699519,789.795338,1703.888476
2,F0003,T0003,T003,MRI,Standard procedure,3731.55,2023-06-28,P131933,D25200,H1082,11455.224800,3,61,2246.888271,2611.032383,519.078775
3,F0004,T0004,T004,MRI,Basic screening,4799.86,2023-09-01,P259179,D41977,H0634,10515.091880,8,88,1993.956559,1358.721884,869.702052
4,F0005,T0005,T005,ECG,Standard procedure,582.05,2023-07-06,P110269,D16372,H0513,17177.983040,1,56,3975.785170,1862.534649,1563.964036
